In [105]:
import pandas as pd
import numpy as np
import pulp as lp
import json
import sys
import os

In [106]:
df = pd.read_csv('data/knp_bytepay_reelect_v1.csv')
df.head()

,session_id,default_pay_mode,bind_card_x1,bytepay_ctr_x1,all_ctr_x1,budget_x1,bind_card_x0,bytepay_ctr_x0,all_ctr_x0,budget_x0
0,202405101320383738a0zb71cy22x3,bp_top,0.000357,0.082155,0.080981,-0.001175,0.000137,0.019261,0.081437,0.062176
1,20240510131620187102za70yb1c22dx,bp_top,0.002636,0.227701,0.795592,0.567891,0.002107,0.223255,0.841327,0.618072
2,20240510205932374969a0bz17yc2d3e,bp_top,0.000692,0.125163,0.191061,0.065897,0.000000,0.000000,0.000000,0.000000
3,20240510084903636354z7ya20xb13w8,bp_top,0.000786,0.101570,0.248613,0.147044,0.000107,0.003223,0.284094,0.280871
4,20240510234744881772az0b71c2dy3e,bp_top,0.002416,0.282996,0.392014,0.109018,0.000410,0.071137,0.419518,0.348381


In [107]:
pv_count = df.shape[0]
print(pv_count)

100


In [109]:
pv_count = df.shape[0]
print(pv_count)

bind_card_x1 = df['bind_card_x1']
bind_card_x0 = df['bind_card_x0']

budget_x1 = df['budget_x1']
budget_x1_pv_rate = np.sum((df['all_ctr_x1'] - df['bytepay_ctr_x1'])) / pv_count
beta = 1 - 0.488
cons_right = beta * pv_count * budget_x1_pv_rate

print('budget_x1_pv_rate: %f' % (budget_x1_pv_rate))
print('cons_right: %f' % (cons_right))
print('beta: %f' % (beta))

100
budget_x1_pv_rate: 0.318123
cons_right: 16.287878
beta: 0.512000


In [110]:
# 求解变量，是否要选取本次为曝光机会
X = [lp.LpVariable(f"x_{i}", 0, 1, lp.LpInteger) for i in range(pv_count)]

# 定义最大化问题
prob = lp.LpProblem("bytepay_reelect", lp.LpMaximize)

# 优化目标
prob += lp.lpSum([(lp.lpDot(bind_card_x1[i], X[i]) + lp.lpDot(bind_card_x0[i], 1 - X[i])) for i in range(len(X))])

# 约束
prob += (lp.lpSum([lp.lpDot(budget_x1[i], X[i]) for i in range(len(X))]) <= beta * pv_count * budget_x1_pv_rate) 

# 求解
status = prob.solve()

#组合结果输出
result = {
    "idx": [],
    "opt": 0.0,
    "opt_x1": 0.0,
    "constraint": 0.0
}

for i in range(len(X)):
    if X[i].value() == 1:
        result["idx"].append("%s" % i)
        result["opt"] += bind_card_x1[i]
        result["opt_x1"] += bind_card_x1[i]
        result["constraint"] += (budget_x1[i])
    else:
        result["opt"] += bind_card_x0[i]

print('idx: %s ' % (str(result['idx'])))
print('idx len: %d ' % (len(result['idx'])))
print('opt: %s ' % (str(result['opt'])))
print('opt_x1: %s ' % (str(result['opt_x1'])))
print('constraint: %s' % (str(result['constraint'])))

print('lambda is %f' % (result['constraint'] - cons_right))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/bytedance/miniconda3/envs/py39_torch2_cpu/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/v7/rktr9bj972l5l321cppqrf340000gn/T/c769715d98cf49568e3f801dce73403f-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/v7/rktr9bj972l5l321cppqrf340000gn/T/c769715d98cf49568e3f801dce73403f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 407 RHS
At line 409 BOUNDS
At line 510 ENDATA
Problem MODEL has 1 rows, 100 columns and 100 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.827197 - 0.00 seconds
Cgl0004I processed model has 1 rows, 96 columns (96 integer (96 of which binary)) and 96 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.171015
Cbc0038I Pass   1: suminf.    0.04353 (1) obj. -0.826959 iterations 

In [94]:
###################################################

In [102]:
df = pd.read_csv('data/knp_bytepay_reelect_v2.csv')
df.head()

,session_id,default_pay_mode,bind_card_x1,bytepay_ctr_x1,all_ctr_x1,budget_x1,bind_card_x0,bytepay_ctr_x0,all_ctr_x0,budget_x0
0,20240510174535752360az0b71c2d3e4,bp_top,0.004192,0.687593,0.748628,0.061035,0.000000,0.000000,0.000000,0.000000
1,20240510165157756864za70by21xc23,bp_top,0.011964,0.290142,0.732599,0.442457,0.000000,0.000000,0.000000,0.000000
2,20240510211531352375za70by12cx32,bp_top,0.000025,0.082892,0.603426,0.520533,0.000007,0.000403,0.647232,0.646828
3,20240510230521907300az07y2xb13cw,bp_top,0.004865,0.568322,0.923259,0.354937,0.001002,0.150113,0.941891,0.791778
4,20240510023229371553z7ya2x0b13c2,bp_top,0.070963,0.623089,0.846899,0.223810,0.009242,0.051975,0.855863,0.803888


In [103]:
pv_count = df.shape[0]
print(pv_count)

bind_card_x1 = df['bind_card_x1']
bind_card_x0 = df['bind_card_x0']

budget_x1 = df['budget_x1']
budget_x1_pv_rate = np.sum((df['all_ctr_x1'] - df['bytepay_ctr_x1'])) / pv_count
beta = 1 - 0.488
cons_right = beta * pv_count * budget_x1_pv_rate

print('beta: %f' % (beta))
print('budget_x1_pv_rate: %f' % (budget_x1_pv_rate))
print('cons_right: %f' % (cons_right))

10000
beta: 0.512000
budget_x1_pv_rate: 0.305588
cons_right: 1564.612052


In [104]:
0.512 * 10000 * 0.305588

1564.61056

In [99]:
# 求解变量，是否要选取本次为曝光机会
X = [lp.LpVariable(f"x_{i}", 0, 1, lp.LpInteger) for i in range(pv_count)]

# 定义最大化问题
prob = lp.LpProblem("bytepay_reelect", lp.LpMaximize)

# 优化目标
prob += lp.lpSum([(lp.lpDot(bind_card_x1[i], X[i]) + lp.lpDot(bind_card_x0[i], 1 - X[i])) for i in range(len(X))])

# 约束
prob += (lp.lpSum([lp.lpDot(budget_x1[i], X[i]) for i in range(len(X))]) <= beta * pv_count * budget_x1_pv_rate) 

# 求解
status = prob.solve()

#组合结果输出
result = {
    "idx": [],
    "opt": 0.0,
    "opt_x1": 0.0,
    "constraint": 0.0
}

for i in range(len(X)):
    if X[i].value() == 1:
        result["idx"].append("%s" % i)
        result["opt"] += bind_card_x1[i]
        result["opt_x1"] += bind_card_x1[i]
        result["constraint"] += (budget_x1[i])
    else:
        result["opt"] += bind_card_x0[i]

print('idx: %s ' % (str(result['idx'])))
print('idx len: %d ' % (len(result['idx'])))
print('opt: %s ' % (str(result['opt'])))
print('opt_x1: %s ' % (str(result['opt_x1'])))
print('constraint: %s' % (str(result['constraint'])))

print('lambda is %f' % (result['constraint'] - cons_right))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/bytedance/miniconda3/envs/py39_torch2_cpu/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/v7/rktr9bj972l5l321cppqrf340000gn/T/aecd25a813c441699a20723c3e8dcc80-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/v7/rktr9bj972l5l321cppqrf340000gn/T/aecd25a813c441699a20723c3e8dcc80-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 40007 RHS
At line 40009 BOUNDS
At line 50010 ENDATA
Problem MODEL has 1 rows, 10000 columns and 10000 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 86.0989 - 0.01 seconds
Cgl0004I processed model has 1 rows, 9594 columns (9594 integer (9594 of which binary)) and 9594 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.17884
Cbc0038I Solution found of -86.0984
Cbc0038I Before 